# EDA

In [ ]:
#%pip install plotly
# %pip install statsmodels

import pandas as pd
import pymysql as mysql
import plotly.express as px
import requests
import json


In [ ]:
conn = mysql.connect(
    host='127.0.0.1',
    user='root',
    password='root',
    database='phonepay'
)
cursor = conn.cursor()

# Aggegrated Insurance table data fetching

cursor.execute("select * from aggregated_insurance")
conn.commit()
table1=cursor.fetchall()

Aggre_insurance = pd.DataFrame(table1,columns=['States','Years','Quarter',
                                               'Name','Transaction_count','Transaction_amount'])

# Aggregated Transaction table data fetching
cursor.execute("select * from aggregated_transaction")
conn.commit()
table2=cursor.fetchall()

Aggre_transaction = pd.DataFrame(table2,columns=['States','Years','Quarter',
                                                 'Transaction_type','Transaction_count','Transaction_amount'])

# Aggregated User table data fetching
cursor.execute("select * from aggregated_user")
conn.commit()
table3=cursor.fetchall()

Aggre_user = pd.DataFrame(table3,columns=['States','Years','Quarter','Brand',
                                          'Transaction_count','Percentage'])

# Map Insurance table data fetching
cursor.execute("select * from map_insurance")
conn.commit()
table4=cursor.fetchall()

Map_insurance = pd.DataFrame(table4,columns=['States','Years','Quarter',
                                             'Districts','Transaction_count','Transaction_amount'])

# Map Transaction table data fetching
cursor.execute("select * from map_transaction")
conn.commit()
table5=cursor.fetchall()

Map_transaction = pd.DataFrame(table5,columns=['States','Years','Quarter',
                                               'Districts','Transaction_count','Transaction_amount'])

# Map User table data fetching
cursor.execute("select * from map_user")
conn.commit()
table6=cursor.fetchall()

Map_user = pd.DataFrame(table6,columns=['States','Years','Quarter',
                                        'Districts','RegisteredUsers','AppOpens'])

# Top Insurance table data fetching
cursor.execute("select * from top_insurance")
conn.commit()
table7=cursor.fetchall()

Top_insurance = pd.DataFrame(table7,columns=['States','Years','Quarter',
                                             'Pincodes','Transaction_count','Transaction_amount'])

# Top Transaction table data fetching
cursor.execute("select * from top_transaction")
conn.commit()
table8=cursor.fetchall()

Top_transaction = pd.DataFrame(table8,columns=['States','Years','Quarter',
                                               'Pincodes','Transaction_count','Transaction_amount'])

# Top User table data fetching
cursor.execute("select * from top_user")
conn.commit()
table9=cursor.fetchall()

Top_user = pd.DataFrame(table9,columns=['States','Years','Quarter',
                                        'Pincodes','RegisteredUsers'])


# Business Case Study

In [ ]:
# Problem 1
cursor.execute("""select States,sum(Transaction_count) as Total_Transaction_count
               from aggregated_transaction
               group by States
               order by Total_Transaction_count desc""")
conn.commit()
query1 = cursor.fetchall()
query1
query1_df = pd.DataFrame(query1,columns=['States','Total_Transaction_count'])
query1_df

fig1 = px.bar(
            query1_df.sort_values("Total_Transaction_count", ascending=True),  # sort for cleaner horizontal bars
            x="Total_Transaction_count",
            y="States",
            orientation="h",
            title="Total Transaction Count in Each State",
            text="Total_Transaction_count",  # show values on bars
            color="Total_Transaction_count",  # add gradient coloring
            color_continuous_scale="Blues"   # color scheme
        )
fig1.show()



In [ ]:
# district wise transaction count in a state uses Map_transaction table
def district_wise_transaction_count(state):
    cursor.execute("""select Districts,States,sum(Transaction_count) as District_Transaction_count
                    from map_transaction
                    where States=%s 
                    group by Districts,States
                    order by District_Transaction_count desc""", (state,))
    conn.commit()
    query2 = cursor.fetchall()
    query2_df = pd.DataFrame(query2,columns=['Districts','States','District_Transaction_count'])
    # using heat map to visualize the data
    fig2 = px.bar(
                query2_df.sort_values("District_Transaction_count", ascending=True),  # sort for cleaner horizontal bars
                x="District_Transaction_count",
                y="Districts",
                orientation="h",
                title=f"District wise Transaction Count in {state}",
                text="District_Transaction_count",  # show values on bars
                color="District_Transaction_count",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme
            )
    fig2.show()
    
    

In [ ]:
district_wise_transaction_count('Maharashtra')

In [ ]:
# yearly transaction_count_analysis Problem 1 query 2 uses aggregated_transaction table
def yearly_transaction_count_analysis(year):
    cursor.execute(f"""select Years,States,sum(Transaction_count) as Yearly_Transaction_count,Quarter
                   from aggregated_transaction
                   where Years={year}
                   group by Years,States,Quarter
                   order by Years,Yearly_Transaction_count desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','States','Yearly_Transaction_count','Quarter'])
    fig = px.bar(
                query_df.sort_values("Yearly_Transaction_count", ascending=True),  # sort for cleaner horizontal bars
                x="Yearly_Transaction_count",
                y="States",
                orientation="h",
                title=f"Yearly Transaction Count in Each State for the year {year}",
                text="Yearly_Transaction_count",  # show values on bars
                color="Yearly_Transaction_count",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme
            )
    fig.show()
    return query_df


In [ ]:
y=yearly_transaction_count_analysis(2020)



In [ ]:
# Quarterly transaction_count_analysis Problem 1 query 3 uses aggregated_transaction table
def quarterly_transaction_count_analysis(year,quarter):
    cursor.execute(f"""select Years,States,sum(Transaction_count) as Quarterly_Transaction_count
                   from aggregated_transaction
                   where Years={year} and Quarter={quarter}
                   group by Years,States
                   order by Years,Quarterly_Transaction_count desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','States','Quarterly_Transaction_count'])
    fig = px.bar(
                query_df.sort_values("Quarterly_Transaction_count", ascending=True),  # sort for cleaner horizontal bars
                x="Quarterly_Transaction_count",
                y="States",
                orientation="h",
                title=f"Quarterly Transaction Count in Each State for the year {year} and quarter {quarter}",
                text="Quarterly_Transaction_count",  # show values on bars
                color="Quarterly_Transaction_count",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme
            )
    fig.show()
    return query_df

In [ ]:
quarterly_transaction_count_analysis(2020,3)

In [ ]:
# Transaction type contribution Problem 1 query 3 uses aggregated_transaction table
# we take Aggre_transaction table because it contains the Transaction_type column

Aggre_transaction['Transaction_type'].unique()

# SELECT Transaction_type, SUM(Transaction_amount) AS total_amount
# FROM aggregated_transaction
# GROUP BY Transaction_type
# ORDER BY total_amount DESC;

# 1st we should show the total transaction amount by transaction type in descending order using bar chart
# 2nd we should show the percentage contribution of each transaction type to the total transaction amount using pie chart
# 3rd we should show the transaction count by transaction type in descending order using bar chart
# 4th we should show the percentage contribution of each transaction type to the total transaction count using pie chart
# 5th we should show the yearly trend of transaction amount by transaction type using line chart
# 6th we should show the Quarterly trend of transaction count by transaction type using line chart

def transaction_type_contribution():
    cursor.execute("""select Transaction_type, SUM(Transaction_amount) AS total_amount
                   from aggregated_transaction
                   group by Transaction_type
                   order by total_amount desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Transaction_type','total_amount'])
    fig = px.bar(
                query_df.sort_values("total_amount", ascending=True),  # sort for cleaner horizontal bars
                x="total_amount",
                y="Transaction_type",
                orientation="h",
                title=f"Total Transaction Amount by Transaction Type",
                text="total_amount",  # show values on bars
                color="total_amount",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme
            )
    fig.show()

    fig2 = px.pie(
                query_df,
                names='Transaction_type',
                values='total_amount',
                title='Percentage Contribution of Each Transaction Type to the Total Transaction Amount'
            )   
    fig2.show()

    cursor.execute("""select Transaction_type, SUM(Transaction_count) AS total_count
                   from aggregated_transaction
                    group by Transaction_type
                    order by total_count desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Transaction_type','total_count'])
    fig3 = px.bar(
                query_df.sort_values("total_count", ascending=True),  # sort for cleaner horizontal bars
                x="total_count",
                y="Transaction_type",
                orientation="h",
                title=f"Total Transaction Count by Transaction Type",
                text="total_count",  # show values on bars
                color="total_count",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme
            )
    fig3.show()

    fig4 = px.pie(
                query_df,
                names='Transaction_type',
                values='total_count',
                title='Percentage Contribution of Each Transaction Type to the Total Transaction Count'
            )   
    fig4.show()

    cursor.execute("""select Years,Transaction_type, SUM(Transaction_amount) AS total_amount
                   from aggregated_transaction
                     group by Years,Transaction_type
                        order by Years, total_amount desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','Transaction_type','total_amount'])
    fig5 = px.line(
                query_df,
                x="Years",
                y="total_amount",
                color='Transaction_type',
                title='Yearly Trend of Transaction Amount by Transaction Type'
            )
    fig5.show()
    cursor.execute("""select Quarter,Transaction_type, SUM(Transaction_count) AS total_count
                   from aggregated_transaction
                     group by Quarter,Transaction_type
                        order by Quarter, total_count desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Quarter','Transaction_type','total_count'])
    fig6 = px.line(
                query_df,
                x="Quarter",
                y="total_count",
                color='Transaction_type',
                title='Quarterly Trend of Transaction Count by Transaction Type'
            )
    fig6.show()

def transaction_type_contribution_Y(year):

    cursor.execute(f"""select Years,Transaction_type, SUM(Transaction_amount) AS total_amount
                   from aggregated_transaction
                   where Years={year}
                     group by Years,Transaction_type
                        order by Years, total_amount desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','Transaction_type','total_amount'])
    fig = px.bar(
                query_df.sort_values("total_amount", ascending=True),  # sort for cleaner horizontal bars
                x="total_amount",
                y="Transaction_type",
                orientation="h",
                title=f"Total Transaction Amount by Transaction Type for the year {year}",
                text="total_amount",  # show values on bars  
                color="total_amount",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme   

    )
    fig.show()

    cursor.execute(f"""select Years,Transaction_type, SUM(Transaction_count) AS total_count
                   from aggregated_transaction
                     where Years={year}
                        group by Years,Transaction_type
                            order by Years, total_count desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','Transaction_type','total_count'])
    fig = px.bar(
                query_df.sort_values("total_count", ascending=True),  # sort for cleaner horizontal bars
                x="total_count",
                y="Transaction_type",
                orientation="h",
                title=f"Total Transaction Count by Transaction Type for the year {year}",
                text="total_count",  # show values on bars
                color="total_count",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme
    )
    fig.show()


In [ ]:

transaction_type_contribution()

transaction_type_contribution_Y(2020)

In [ ]:
# Fastest growing state (YoY growth) for Problem 1 query 4 uses aggregated_transaction table

# 1st we should show the growth % for years for the transaction amount for years using bar chart
# 2nd we should show the growth % for years for the transaction count for years using bar chart
# 3rd we should show the growth % for each state for the transaction amount using bar chart
# 4th we should show the growth % for each state for the transaction count using bar chart

def fastest_growing_state_yoy_growth():
    cursor.execute("""select Years, 
                   (SUM(Transaction_amount) - LAG(SUM(Transaction_amount)) OVER (ORDER BY Years)) / LAG(SUM(Transaction_amount)) OVER (ORDER BY Years) * 100 AS growth_percentage_amount
                   from aggregated_transaction
                   group by Years
                   order by Years""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','growth_percentage_amount'])
    fig = px.bar(
                query_df.sort_values("growth_percentage_amount", ascending=True),  # sort for cleaner horizontal bars
                x="growth_percentage_amount",
                y="Years",
                orientation="h",
                title=f"Yearly Growth Percentage of Transaction Amount",
                text="growth_percentage_amount",  # show values on bars
                color="growth_percentage_amount",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme
            )
    
    fig.show()


    # total transaction_amount for every year
    cursor.execute(""" Select Years,
                   sum(Transaction_amount) as Total_Trans_amt
                   from aggregated_transaction
                   group by Years
                   order by Years""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','Total_Trans_amt'])
    fig = px.bar(
                query_df.sort_values("Total_Trans_amt", ascending=True),  # sort for cleaner horizontal bars
                x="Total_Trans_amt",
                y="Years",
                orientation="h",
                title=f"Yearly Growth of Transaction Amount",
                text="Total_Trans_amt",  # show values on bars
                color="Total_Trans_amt",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme
            )
          
    fig.show()

    # growth percentage of transaction amount for each state
def fastest_growing_state_yoy_growth_state(state):
    cursor.execute(f"""select States,Years,
                   (SUM(Transaction_amount) - LAG(SUM(Transaction_amount)) OVER (PARTITION BY States ORDER BY Years)) / LAG(SUM(Transaction_amount)) OVER (PARTITION BY States ORDER BY Years) * 100 AS growth_percentage_amount
                   from aggregated_transaction
                   where States='{state}'
                   group by States,Years
                   order by States,Years""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['States','Years','growth_percentage_amount'])
    fig = px.bar(
                query_df.sort_values("growth_percentage_amount", ascending=True),  # sort for cleaner horizontal bars
                x="growth_percentage_amount",
                y="Years",
                orientation="h",
                title=f"Yearly Growth Percentage of Transaction Amount in {state}",
                text="growth_percentage_amount",  # show values on bars
                color="growth_percentage_amount",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme
            )
    fig.show()
    
    # total transaction_amount for every state
    cursor.execute(f""" Select States,Years,
                   sum(Transaction_amount) as Total_Trans_amt
                   from aggregated_transaction
                   where States='{state}'
                   group by States,Years
                   order by States,Years""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['States','Years','Total_Trans_amt'])
    fig = px.bar(
                query_df.sort_values("Total_Trans_amt", ascending=True),  # sort for cleaner horizontal bars
                x="Total_Trans_amt",
                y="Years",
                orientation="h",
                title=f"Yearly Total Transaction Amount in {state}",
                text="Total_Trans_amt",  # show values on bars
                color="Total_Trans_amt",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme
            )
    fig.show()


# Yoy growth transaction amount for overall states among all states
def fastest_growing_state_yoy_growth_overall_states():
    cursor.execute(f"""select States,
                   (SUM(Transaction_amount) - LAG(SUM(Transaction_amount)) OVER (PARTITION BY States ORDER BY Years)) / LAG(SUM(Transaction_amount)) OVER (PARTITION BY States ORDER BY Years) * 100 AS growth_percentage_amount
                   from aggregated_transaction
                   group by States,Years
                   order by States,Years""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['States','growth_percentage_amount'])
    fig = px.bar(
                query_df.sort_values("growth_percentage_amount", ascending=True),  # sort for cleaner horizontal bars
                x="growth_percentage_amount",
                y="States",
                orientation="h",
                title=f"Yearly Growth Percentage of Transaction Amount for Overall States",
                text="growth_percentage_amount",  # show values on bars
                color="growth_percentage_amount",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme
            )
    fig.show()
    return query_df




In [ ]:
fastest_growing_state_yoy_growth()

In [ ]:
fastest_growing_state_yoy_growth_state( 'Maharashtra')

In [ ]:
fastest_growing_state_yoy_growth_overall_states()


In [ ]:
# Display of all states with only declining transaction amount Problem 1 query 5 uses aggregated_transaction table in bar chart
# 1st we need to take the name of states which have declining transaction amount for all years and return the names of those states
# 2nd we need to show the states with declining transaction amount in bar chart for selected year 

def declining_transaction_amount_states():
    cursor.execute(f"""WITH state_yearly AS (
    SELECT 
        States,
        Years,
        SUM(Transaction_amount) AS total_amt
    FROM aggregated_transaction
    GROUP BY States, Years
),
with_lag AS (
    SELECT 
        States,
        Years,
        total_amt,
        LAG(total_amt) OVER (PARTITION BY States ORDER BY Years) AS prev_amt
    FROM state_yearly
)
SELECT DISTINCT States
FROM with_lag
WHERE total_amt < prev_amt
ORDER BY States;""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['States'])
    return query_df['States'].unique().tolist()


# line chart for the states with declining transaction amount year wise
def declining_transaction_amount_states_yearwise(state):
    cursor.execute(f""" select States,Years,sum(Transaction_amount) as Total_Transaction_amount
                   from aggregated_transaction
                   where States='{state}'
                   group by States,Years
                   order by Years""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['States','Years','Total_Transaction_amount'])
     # Convert Decimal → float
    query_df['Total_Transaction_amount'] = query_df['Total_Transaction_amount'].astype(float)
    query_df['Amount_Billions'] = query_df['Total_Transaction_amount'] / 1e9  # Convert to billions
    fig = px.line(
                query_df,
                x='Years',
                y='Amount_Billions',
                title=f'Transaction Amount Trend: {state} (in Billions ₹)',
                markers=True,
                line_shape='linear'
            )
    fig.show()



In [ ]:
def declining_transaction_amount_states():
    cursor.execute(f"""WITH state_yearly AS (
    SELECT 
        States,
        Years,
        SUM(Transaction_amount) AS total_amt
    FROM aggregated_transaction
    GROUP BY States, Years
),
with_lag AS (
    SELECT 
        States,
        Years,
        total_amt,
        LAG(total_amt) OVER (PARTITION BY States ORDER BY Years) AS prev_amt
    FROM state_yearly
)
SELECT DISTINCT States
FROM with_lag
WHERE total_amt < prev_amt
ORDER BY States;""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['States'])
    return query_df['States'].unique().tolist()

In [ ]:
states =declining_transaction_amount_states()
states

In [ ]:
declining_transaction_amount_states_yearwise('Tamil Nadu')

## Probelm 2 Device Dominance and User Engagement

In [ ]:
Aggre_user

In [ ]:
# -- Q1: Registered users by brand
# 1st we should show the total registered users by brand in descending order using bar chart
# 2nd we should show the percentage contribution of each brand to the total registered users using pie chart for top 5 brands
# 3rd we should show the yearly trend of registered users by brand using line chart
# 4th we should show the YoY of selected brand

def registered_users_by_brand():
    cursor.execute("""select Brands, SUM(Transaction_count) AS total_registered_users
                   from aggregated_user
                   group by Brands
                   order by total_registered_users desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Brands','total_registered_users'])
    fig = px.bar(
                query_df.sort_values("total_registered_users", ascending=True),  # sort for cleaner horizontal bars
                x="total_registered_users",
                y="Brands",
                orientation="h",
                title=f"Total Registered Users by Brand",
                text="total_registered_users",  # show values on bars
                color="total_registered_users",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme
            )
    fig.show()

    fig2 = px.pie(
                query_df.head(5),
                names='Brands',
                values='total_registered_users',
                title='Percentage Contribution of Each Brand to the Total Registered Users (Top 5 Brands)'
            )
    fig2.show()

    cursor.execute("""select Years,Brands, SUM(Transaction_count) AS total_registered_users
                   from aggregated_user
                   group by Years,Brands
                   order by Years, total_registered_users desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','Brands','total_registered_users'])
    fig3 = px.line(
                query_df,
                x="Years",
                y="total_registered_users",
                color='Brands',
                title='Yearly Trend of Registered Users by Brand'
            )
    fig3.show()

# 4th we should show the YoY of selected brand
def registered_users_by_brand_YOY(brand):
    cursor.execute(f"""select Years,Brands, SUM(Transaction_count) AS total_registered_users
                   from aggregated_user
                   where Brands='{brand}'
                   group by Years,Brands
                   order by Years, total_registered_users desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','Brands','total_registered_users'])
    fig = px.bar(
                query_df.sort_values("total_registered_users", ascending=True),  # sort for cleaner horizontal bars
                x="total_registered_users",
                y="Years",
                orientation="h",
                title=f"Yearly Registered Users for the Brand {brand}",
                text="total_registered_users",  # show values on bars
                color="total_registered_users",  # add gradient coloring
                color_continuous_scale="Blues"   # color scheme
            )
    fig.show()


In [ ]:
registered_users_by_brand()

In [ ]:
registered_users_by_brand_YOY()

In [ ]:
# problem 2 query 2
# Most engaged brand (highest %)
# 1st we should show the overall percentage of each brand to the total registered users using bar chart
# 2nd we should show the yearly trend of percentage of each brand to the total registered users using line chart
# 3rd from the year we should show the state wise percentage of each brand to the total registered users using bar chart

def most_engaged_brand_highest_percentage():
    cursor.execute("""select Brands, SUM(Percentage) AS total_percentage
                   from aggregated_user
                   group by Brands
                   order by total_percentage desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Brands','total_percentage'])
    fig = px.bar(
                query_df.sort_values("total_percentage", ascending=True),  # sort for cleaner horizontal bars
                x="total_percentage",
                y="Brands",
                orientation="h",
                title=f"Overall Percentage of Each Brand ",
                text="total_percentage",  # show values on bars
                color="total_percentage",  # add gradient coloring
            )
    fig.show()

def most_engaged_brand_highest_percentage_yearly(year):
    cursor.execute(f"""select Years,Brands, SUM(Percentage) AS total_percentage
                   from aggregated_user
                   where Years={year}
                   group by Years,Brands
                   order by Years, total_percentage desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','Brands','total_percentage'])
    fig = px.bar(
                query_df.sort_values("total_percentage", ascending=True),  # sort for cleaner horizontal bars
                x="total_percentage",
                y="Brands",
                orientation="h",
                title=f"{year} Percentage of Each Brand ",
                text="total_percentage",  # show values on bars
                color="total_percentage",  # add gradient coloring
            )
    fig.show()
    return query_df

def most_engaged_brand_highest_percentage_state(year,state):
    cursor.execute(f"""select States,Years,Brands, SUM(Percentage) AS total_percentage
                   from aggregated_user
                   where Years={year} and States='{state}'
                   group by States,Years,Brands
                   order by States,Years, total_percentage desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['States','Years','Brands','total_percentage'])
    fig = px.bar(
                query_df.sort_values("total_percentage", ascending=True),  # sort for cleaner horizontal bars
                x="total_percentage",
                y="Brands",
                orientation="h",
                title=f"State wise Percentage  for the year {year}",
                text="total_percentage",  # show values on bars
                color="total_percentage",  # add gradient coloring
            )
    fig.show()
    return query_df





In [ ]:
most_engaged_brand_highest_percentage_state(2020,'Maharashtra')

In [ ]:
a=most_engaged_brand_highest_percentage_yearly(2021)


In [ ]:
most_engaged_brand_highest_percentage()

In [ ]:
# problem 2 query 3
# Brand share by state
# 1st we should show the overall percentage of each brand to the total registered users using bar chart
# 2nd we should show the yearly trend of percentage of each brand to the total registered users using line chart

def brand_share_by_state():
    cursor.execute("""select Brands, SUM(Percentage) AS total_percentage
                   from aggregated_user
                   group by Brands
                   order by total_percentage desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Brands','total_percentage'])
    fig = px.bar(
                query_df.sort_values("total_percentage", ascending=True),  # sort for cleaner horizontal bars
                x="total_percentage",
                y="Brands",
                orientation="h",
                title=f"Overall Percentage of Each Brand ",
                text="total_percentage",  # show values on bars
                color="total_percentage",  # add gradient coloring
            )
    fig.show()

    cursor.execute("""select Years,Brands, SUM(Percentage) AS total_percentage
                   from aggregated_user
                   group by Years,Brands
                   order by Years, total_percentage desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','Brands','total_percentage'])
    fig = px.line(
                query_df,
                x="Years",
                y="total_percentage",
                color='Brands',
                title='Yearly Trend of Percentage of Each Brand'
            )
    fig.show()

In [ ]:
brand_share_by_state()

In [ ]:
Aggre_user

In [ ]:
# problem 2 query 4
# Top device in each quarter
#  1st for selected year we should show the top device in each quarter using bar chart using transaction count and seperate bar cgart for top 5 brands
# 2nd we should show the top device in each quarter using bar chart
# 3rd we should show the yearly trend of percentage of each brand to the total registered users using line chart

 # ✅ Create human-readable format
def human_format(num):
        if num is None: 
            return None
        for unit in ['', 'K', 'M', 'B', 'T']:
            if abs(num) < 1000:
                return f"{num:.2f}{unit}"
            num /= 1000
        return f"{num:.2f}P"

def top_device_in_each_quarter_year(year):
    cursor.execute(f"""select Years,Quarter,Brands, SUM(Transaction_count) AS Transaction_count
                   from aggregated_user
                   where Years={year}
                   group by Years,Quarter,Brands
                   order by Years,Quarter, Transaction_count desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','Quarter','Brands','Transaction_count'])
    query_df['Transaction_count_fmt'] = query_df['Transaction_count'].apply(human_format)
    fig = px.bar(
                query_df.sort_values("Transaction_count", ascending=True),  # sort for cleaner horizontal bars
                x="Transaction_count",
                y="Brands",
                orientation="h",
                title=f"Top Device in Each Quarter for the year {year}",
                text="Transaction_count_fmt",  # show values on bars
                color="Transaction_count",  # add gradient coloring
            )
    fig.show()

    # top 5 brands
    top_5_brands = query_df['Brands'].value_counts().nlargest(5).index
    top_5_df = query_df[query_df['Brands'].isin(top_5_brands)]
    fig2 = px.bar(
                top_5_df.sort_values("Transaction_count", ascending=True),
                x="Transaction_count",
                y="Brands",
                orientation="h",
                title=f"Top 5 Devices in Each Quarter for the year {year}",
                text="Transaction_count_fmt",  # show values on bars
                color="Transaction_count",
            )
    fig2.show()


# 2nd we should show the top device in given quarter for given year using bar chart
def top_device_in_given_quarter(year, quarter):
    cursor.execute(f"""select Years,Quarter,Brands, SUM(Transaction_count) AS Transaction_count
                   from aggregated_user
                   where Years={year} and Quarter={quarter}
                   group by Years,Quarter,Brands
                   order by Years,Quarter, Transaction_count desc""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','Quarter','Brands','Transaction_count'])
    query_df['Transaction_count_fmt'] = query_df['Transaction_count'].apply(human_format)
    fig = px.bar(
                query_df.sort_values("Transaction_count", ascending=True),  # sort for cleaner horizontal bars
                x="Transaction_count",
                y="Brands",
                orientation="h",
                title=f"Top Device in Each Quarter",
                text="Transaction_count_fmt",  # show values on bars
                color="Transaction_count",  # add gradient coloring
            )
    fig.show()
    # top 5 brands
    top_5_brands = query_df['Brands'].value_counts().nlargest(5).index
    top_5_df = query_df[query_df['Brands'].isin(top_5_brands)]
    fig2 = px.bar(
                top_5_df.sort_values("Transaction_count", ascending=True),
                x="Transaction_count",
                y="Brands",
                orientation="h",
                title=f"Top 5 Devices in Each Quarter for the year {year}",
                text="Transaction_count_fmt",  # show values on bars
                color="Transaction_count",
            )
    fig2.show()


    


In [ ]:
top_device_in_each_quarter_year(2020)

In [ ]:
top_device_in_given_quarter(2020,3)

In [ ]:
# problem 2 query 5
# Underutilized devices

def underutilized_devices():
    cursor.execute(f""" select Brands,sum(Transaction_count) as registered_users,
                   round(avg(Percentage)*100,2) as avg_useage
                   from aggregated_user
                   group by brands
                   having avg_useage <5""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['Brands','registered_users','avg_useage'])
    fig=px.bar(
                query_df.sort_values("avg_useage", ascending=True),  # sort for cleaner horizontal bars
                x="avg_useage",
                y="Brands",
                orientation="h",
                title=f"Underutilized Devices (Average Usage < 5%)",
                text="avg_useage",  # show values on bars
                color="avg_useage",  # add gradient coloring
            )
    fig.show()


In [ ]:
underutilized_devices()

# Problem 3 3. Insurance Penetration and Growth Potential

In [ ]:
Aggre_insurance

In [ ]:
# Problem 3 query 1
# State-wise insurance penetration
# 1st we should show the state wise insurance count and insurance amount on seperate bar chart
# 2nd we should show the top 5 high and low insurance amount states using bar chart
# 3rd we should show the top 5 high and low insurance count states using bar chart

def state_wise_insurance_penetration():
    cursor.execute(f""" select States,sum(Insurance_count) as Total_Insurance_count,
                     sum(TotalIns_amount) as Total_Insurance_amount
                     from aggregated_insurance
                     group by States
                     order by Total_Insurance_count desc""")
    conn.commit()
    query=cursor.fetchall()

    State_wise_insurance = pd.DataFrame(query,columns=['States','Total_Insurance_count','Total_Insurance_amount'])
    # Convert numeric columns to correct dtype
    State_wise_insurance['Total_Insurance_count'] = pd.to_numeric(State_wise_insurance['Total_Insurance_count'], errors='coerce')
    State_wise_insurance['Total_Insurance_amount'] = pd.to_numeric(State_wise_insurance['Total_Insurance_amount'], errors='coerce')
    fig1 = px.bar(
                State_wise_insurance.sort_values("Total_Insurance_count", ascending=True),  # sort for cleaner horizontal bars
                x="Total_Insurance_count",
                y="States",
                orientation="h",
                title='State Wise Insurance Count',
                color_discrete_sequence=px.colors.sequential.Viridis
    )
    fig1.show()
    fig2 = px.bar(
                State_wise_insurance.sort_values("Total_Insurance_amount", ascending=True),  # sort for cleaner horizontal bars
                x="Total_Insurance_amount",
                y="States",
                orientation="h",
                title='State Wise Insurance Amount',
                color_discrete_sequence=px.colors.sequential.Viridis
    )
    fig2.show()

    # Top 5 high and low insurance amount states
    top_5_high_amount = State_wise_insurance.nlargest(5, 'Total_Insurance_amount')
    top_5_low_amount = State_wise_insurance.nsmallest(5, 'Total_Insurance_amount')

    fig3 = px.bar(top_5_high_amount,x='States',y='Total_Insurance_amount',title='Top 5 States by Insurance Amount',
                  color_discrete_sequence=px.colors.sequential.Viridis)
    fig3.show()

    fig4 = px.bar(top_5_low_amount,x='States',y='Total_Insurance_amount',title='Bottom 5 States by Insurance Amount',
                  color_discrete_sequence=px.colors.sequential.Bluered_r)
    fig4.show()

    # Top 5 high and low insurance count states
    top_5_high_count = State_wise_insurance.nlargest(5, 'Total_Insurance_count')
    top_5_low_count = State_wise_insurance.nsmallest(5, 'Total_Insurance_count')

    fig5 = px.bar(top_5_high_count,x='States',y='Total_Insurance_count',title='Top 5 States by Insurance Count',
                  color_discrete_sequence=px.colors.sequential.Viridis)
    fig5.show()

    fig6 = px.bar(top_5_low_count,x='States',y='Total_Insurance_count',title='Bottom 5 States by Insurance Count',
                  color_discrete_sequence=px.colors.sequential.Bluered_r)
    fig6.show()

In [ ]:
state_wise_insurance_penetration()

In [ ]:
# problem 3 query 2
# Quarterly growth trend
# 1st we should should show the year wise growth on transaction amount and count for selected state using bar chart
# 2nd we should show the quarter wise growth on transaction amount and count for selected state using bar chart
# 3rd we should show the quarterly trend of transaction amount and count for selected state using line chart

def quarterly_growth_trend_state(state):
    cursor.execute(f""" select Years, sum(TotalIns_amount) as Total_Insurance_amount,
                   (sum(TotalIns_amount) - LAG(sum(TotalIns_amount)) OVER (ORDER BY Years)) / LAG(sum(TotalIns_amount)) OVER (ORDER BY Years) * 100 AS growth_percentage_amount
                   from aggregated_insurance
                   where States='{state}'
                   group by Years
                   order by Years""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['Years','Total_Insurance_amount','growth_percentage_amount'])
    fig1 = px.bar(
                query_df.sort_values("growth_percentage_amount", ascending=True),  # sort for cleaner horizontal bars
                x="growth_percentage_amount",
                y="Years",
                orientation="h",
                title=f'Yearly Growth Percentage of Insurance Amount in {state}',
                text="growth_percentage_amount",  # show values on bars
                color="growth_percentage_amount",  # add gradient coloring
            )
    fig1.show()
    fig2 = px.bar(
                query_df.sort_values("Total_Insurance_amount", ascending=True),  # sort for cleaner horizontal bars
                x="Total_Insurance_amount",
                y="Years",
                orientation="h",
                title=f'Yearly Total Insurance Amount in {state}',
                text="Total_Insurance_amount",  # show values on bars
                color="Total_Insurance_amount",  # add gradient coloring
            )
    fig2.show()

    cursor.execute(f""" select Quarter, sum(TotalIns_amount) as Total_Insurance_amount,
                   (sum(TotalIns_amount) - LAG(sum(TotalIns_amount)) OVER (ORDER BY Quarter)) / LAG(sum(TotalIns_amount)) OVER (ORDER BY Quarter) * 100 AS growth_percentage_amount
                   from aggregated_insurance
                   where States='{state}'
                   group by Quarter
                   order by Quarter""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['Quarter','Total_Insurance_amount','growth_percentage_amount'])
    fig3 = px.bar(
                query_df.sort_values("growth_percentage_amount", ascending=True),  # sort for cleaner horizontal bars
                x="growth_percentage_amount",
                y="Quarter",
                orientation="h",
                title=f'Quarterly Growth Percentage of Insurance Amount in {state}',
                text="growth_percentage_amount",  # show
                color="growth_percentage_amount"
    )
    fig3.show()


In [ ]:
quarterly_growth_trend_state('Maharashtra')

In [ ]:
# problem 3 query 3
# States with highest penetration 
# SELECT States,round(SUM(TotalIns_amount)/SUM(Insurance_count),2) AS avg_premium
# FROM aggregated_insurance
# GROUP BY States
# ORDER BY avg_premium DESC;


def states_with_highest_penetration():
    cursor.execute(f""" select States,round(SUM(TotalIns_amount)/SUM(Insurance_count),2) AS avg_premium
                   FROM aggregated_insurance
                   GROUP BY States
                   ORDER BY avg_premium DESC""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','avg_premium'])
    fig = px.bar(
                query_df.sort_values("avg_premium", ascending=True),  # sort for cleaner horizontal bars
                x="avg_premium",
                y="States",
                orientation="h",
                title=f"States with Highest Penetration (Average Premium)",
                text="avg_premium",  # show values on bars
                color="avg_premium",  # add gradient coloring
            )
    fig.show()

In [ ]:
states_with_highest_penetration()

In [ ]:
# problem 3 query 4
# Fastest growing state
# 1st we should show the yearly growth percentage of insurance amount for overall states among all states using bar chart
# 2nd we should show the yearly growth percentage of insurance amount for selected state using bar chart
def fastest_growing_state():
    cursor.execute(f""" select Years, 
                   (SUM(TotalIns_amount) - LAG(SUM(TotalIns_amount)) OVER (ORDER BY Years)) / LAG(SUM(TotalIns_amount)) OVER (ORDER BY Years) * 100 AS growth_percentage_amount
                   from aggregated_insurance
                   group by Years
                   order by Years""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['Years','growth_percentage_amount'])
    fig = px.bar(
                query_df.sort_values("growth_percentage_amount", ascending=True),  # sort for cleaner horizontal bars
                x="growth_percentage_amount",
                y="Years",
                orientation="h",
                title=f"Yearly Growth Percentage of Insurance Amount",
                text="growth_percentage_amount",  # show values on bars
                color="growth_percentage_amount",  # add gradient coloring
            )
    fig.show()
    return query_df
def fastest_growing_state_selected(state):
    cursor.execute(f""" select States,Years,
                   (SUM(TotalIns_amount) - LAG(SUM(TotalIns_amount)) OVER (PARTITION BY States ORDER BY Years)) / LAG(SUM(TotalIns_amount)) OVER (PARTITION BY States ORDER BY Years) * 100 AS growth_percentage_amount
                   from aggregated_insurance
                   where States='{state}'
                   group by States,Years
                   order by States,Years""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','Years','growth_percentage_amount'])
    fig = px.bar(
                query_df.sort_values("growth_percentage_amount", ascending=True),  # sort for cleaner horizontal bars
                x="growth_percentage_amount",
                y="Years",
                orientation="h",
                title=f"Yearly Growth Percentage of Insurance Amount in {state}",
                text="growth_percentage_amount",  # show values on bars
                color="growth_percentage_amount",  # add gradient coloring
            )
    fig.show()


In [ ]:
fastest_growing_state()

In [ ]:
fastest_growing_state_selected('Maharashtra')

In [ ]:
# problem 3 query 5
# Untapped states (low insurance count)
def untapped_states_low_insurance_count():
    cursor.execute(f""" select States,sum(Insurance_count) as Total_Insurance_count
                   from aggregated_insurance
                   group by States
                   having Total_Insurance_count < 50000
                   order by Total_Insurance_count""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','Total_Insurance_count'])
    fig = px.bar(
                query_df.sort_values("Total_Insurance_count", ascending=True),  # sort for cleaner horizontal bars
                x="Total_Insurance_count",
                y="States",
                orientation="h",
                title=f"Untapped States (Low Insurance Count < 50,000)",
                text="Total_Insurance_count",  # show values on bars
                color="Total_Insurance_count",  # add gradient coloring
            )
    fig.show()

In [ ]:
untapped_states_low_insurance_count()

# Problem 4 

In [ ]:
Aggre_transaction

In [ ]:
Map_transaction

In [ ]:
# problem 4 query 1
# State contribution share
# SELECT States, SUM(Transaction_amount) AS total_amt,
#        SUM(Transaction_amount) * 100 / sum(sum(Transaction_amount)) over() percent_share
# FROM aggregated_transaction
# GROUP BY States;


def state_contribution_share():
    cursor.execute(f""" select States, SUM(Transaction_amount) AS total_amt,
                   SUM(Transaction_amount) * 100 / sum(sum(Transaction_amount)) over() percent_share
                   FROM aggregated_transaction
                   GROUP BY States
                   order by total_amt desc""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','total_amt','percent_share'])
    fig = px.bar(
                query_df.sort_values("percent_share", ascending=True),  # sort for cleaner horizontal bars
                x="percent_share",
                y="States",
                orientation="h",
                title=f"State Contribution Share",
                text="percent_share",  # show values on bars
                color="percent_share",  # add gradient coloring
            )
    fig.show()
    query_df['percent_share'] = pd.to_numeric(query_df['percent_share'],errors='coerce')
    top_5_states = query_df.nlargest(5, 'percent_share')
    fig2 = px.bar(top_5_states,x='States',y='percent_share',title='Top 5 States by Contribution Share',
                  color_discrete_sequence=px.colors.sequential.Viridis)
    fig2.show()
    bottom_5_states = query_df.nsmallest(5, 'percent_share')
    fig3 = px.bar(bottom_5_states,x='States',y='percent_share',title='Bottom 5 States by Contribution Share',
                    color_discrete_sequence=px.colors.sequential.Bluered_r) 
    fig3.show()

In [ ]:
state_contribution_share()

In [ ]:
# problem 4 query 2
# Districts with highest transaction count
# 1st we should show the top 10 districts with highest transaction count using bar chart from selected state
# 2nd we should show the top 10 districts with lowest transaction count using bar chart from selected state
# 3rd for selected year we should show the top 10 districts with highest transaction count using bar chart from selected state
# 4th for selected year we should show the top 10 districts with lowest transaction count using bar chart from selected state

def districts_with_highest_transaction_count(state):
    cursor.execute(f""" select States,Districts, SUM(Transaction_count) AS total_transaction_count
                   from map_transaction
                   where States='{state}'
                   group by Districts,States
                   order by total_transaction_count desc""")
    conn.commit()
    query=cursor.fetchall()
    
    query_df=pd.DataFrame(query,columns=['States','Districts','total_transaction_count'])
    query_df['total_transaction_count'] = pd.to_numeric(query_df['total_transaction_count'],errors='coerce')
    fig = px.bar(
                query_df.sort_values("total_transaction_count", ascending=True),  # sort for cleaner horizontal bars
                x="total_transaction_count",
                y="Districts",
                orientation="h",
                title=f"Districts with Highest Transaction Count on {state}",
                text="total_transaction_count",  # show values on bars
                color="total_transaction_count",  # add gradient coloring
            )
    fig.show()
    top_10_districts = query_df.nlargest(10, 'total_transaction_count')
    fig2 = px.bar(top_10_districts,x='Districts',y='total_transaction_count',title=f'Top 10 Districts by Transaction Count on {state}',
                  color_discrete_sequence=px.colors.sequential.Viridis)
    fig2.show()
    bottom_10_districts = query_df.nsmallest(10, 'total_transaction_count')
    fig3 = px.bar(bottom_10_districts,x='Districts',y='total_transaction_count',title=f'Bottom 10 Districts by Transaction Count on {state}',
                    color_discrete_sequence=px.colors.sequential.Bluered_r) 
    fig3.show()
    return query_df
def districts_with_highest_transaction_count_year(year,state):
    cursor.execute(f""" select States,Districts, SUM(Transaction_count) AS total_transaction_count
                   from map_transaction
                   where Years={year} and States='{state}'
                   group by Districts,States
                   order by total_transaction_count desc""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','Districts','total_transaction_count'])
    query_df['total_transaction_count'] = pd.to_numeric(query_df['total_transaction_count'],errors='coerce')

    fig = px.bar(
                query_df.sort_values("total_transaction_count", ascending=True),  # sort for cleaner horizontal bars
                x="total_transaction_count",
                y="Districts",
                orientation="h",
                title=f"Districts with Highest Transaction Count in {year}",
                text="total_transaction_count",  # show values on bars
                color="total_transaction_count",  # add gradient coloring
            )
    fig.show()
    top_10_districts = query_df.nlargest(10, 'total_transaction_count')
    fig2 = px.bar(top_10_districts,x='Districts',y='total_transaction_count',title=f'Top 10 Districts by Transaction Count in {year}',
                  color_discrete_sequence=px.colors.sequential.Viridis)
    fig2.show()
    bottom_10_districts = query_df.nsmallest(10, 'total_transaction_count')
    fig3 = px.bar(bottom_10_districts,x='Districts',y='total_transaction_count',title=f'Bottom 10 Districts by Transaction Count in {year}',
                    color_discrete_sequence=px.colors.sequential.Bluered_r) 
    fig3.show()

In [ ]:
districts_with_highest_transaction_count('Maharashtra')


In [ ]:
districts_with_highest_transaction_count_year(2021,'Maharashtra')

In [ ]:
# problem 4 query 3
# Yearly growth rate
# 1st we should show the yearly growth rate of transaction amount for overall states among all states using bar chart and also top 5 states
# 2nd we should show the yearly growth rate of transaction amount for selected state using bar chart

def yearly_growth_rate():
    cursor.execute(f""" select Years, 
                   (SUM(Transaction_amount) - LAG(SUM(Transaction_amount)) OVER (ORDER BY Years)) / LAG(SUM(Transaction_amount)) OVER (ORDER BY Years) * 100 AS growth_percentage_amount
                   from aggregated_transaction
                   group by Years
                   order by Years""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['Years','growth_percentage_amount'])
    fig = px.bar(
                query_df.sort_values("growth_percentage_amount", ascending=True),  # sort for cleaner horizontal bars
                x="growth_percentage_amount",
                y="Years",
                orientation="h",
                title=f"Yearly Growth Percentage of Transaction Amount",
                text="growth_percentage_amount",  # show values on bars
                color="growth_percentage_amount",  # add gradient coloring
            )
    fig.show()
    # top 5 states
    cursor.execute(f""" select States,Years,
                   (SUM(Transaction_amount) - LAG(SUM(Transaction_amount)) OVER (PARTITION BY States ORDER BY Years)) / LAG(SUM(Transaction_amount)) OVER (PARTITION BY States ORDER BY Years) * 100 AS growth_percentage_amount
                   from aggregated_transaction
                   group by States,Years
                   order by States,Years""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','Years','growth_percentage_amount'])
    query_df['growth_percentage_amount'] = pd.to_numeric(query_df['growth_percentage_amount'],errors='coerce')
    top_5_states = query_df.nlargest(5, 'growth_percentage_amount')     
    fig2 = px.bar(top_5_states,x='States',y='growth_percentage_amount',title='Top 5 States by Growth Percentage',
                  color_discrete_sequence=px.colors.sequential.Viridis)
    fig2.show()
    return query_df 
def yearly_growth_rate_selected(state):
    cursor.execute(f""" select States,Years,
                   (SUM(Transaction_amount) - LAG(SUM(Transaction_amount)) OVER (PARTITION BY States ORDER BY Years)) / LAG(SUM(Transaction_amount)) OVER (PARTITION BY States ORDER BY Years) * 100 AS growth_percentage_amount
                   from aggregated_transaction
                   where States='{state}'
                   group by States,Years
                   order by States,Years""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','Years','growth_percentage_amount'])
    fig = px.bar(
                query_df.sort_values("growth_percentage_amount", ascending=True),  # sort for cleaner horizontal bars
                x="growth_percentage_amount",
                y="Years",
                orientation="h",
                title=f"Yearly Growth Percentage of Transaction Amount in {state}",
                text="growth_percentage_amount",  # show values on bars
                color="growth_percentage_amount",  # add gradient coloring
            )
    fig.show()

In [ ]:
yearly_growth_rate()

In [ ]:
yearly_growth_rate_selected('Maharashtra')

In [ ]:
# problem 4 query 4
# Quarter with peak transactions

def quarter_with_peak_transactions():
    cursor.execute(f""" select Quarter, SUM(Transaction_amount) AS total_transaction_amount
                   from aggregated_transaction
                   group by Quarter
                   order by total_transaction_amount desc""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['Quarter','total_transaction_amount'])
    fig = px.bar(
                query_df.sort_values("total_transaction_amount", ascending=True),  # sort for cleaner horizontal bars
                x="total_transaction_amount",
                y="Quarter",
                orientation="h",
                title=f"Quarter with Peak Transactions",
                text="total_transaction_amount",  # show values on bars
                color="total_transaction_amount",  # add gradient coloring
            )
    fig.show()
    return query_df
# state wise peak quarter
def quarter_with_peak_transactions_state(state):
    cursor.execute(f""" select States,Quarter, SUM(Transaction_amount) AS total_transaction_amount
                   from aggregated_transaction
                   where States='{state}'
                   group by States,Quarter
                   order by total_transaction_amount desc""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','Quarter','total_transaction_amount'])
    fig = px.bar(
                query_df.sort_values("total_transaction_amount", ascending=True),  # sort for cleaner horizontal bars
                x="total_transaction_amount",
                y="Quarter",
                orientation="h",
                title=f"Quarter with Peak Transactions in {state}",
                text="total_transaction_amount",  # show values on bars
                color="total_transaction_amount",  # add gradient coloring
            )
    fig.show()
    return query_df

In [ ]:
# problem 4 query 5
# Compare top vs bottom states
# 1st show the top 5 states
# 2nd show the bottom 5 states
def compare_top_vs_bottom_states():
    cursor.execute(f""" select States, SUM(Transaction_amount) AS total_transaction_amount
                   from aggregated_transaction
                   group by States
                   order by total_transaction_amount desc""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','total_transaction_amount'])
    query_df['total_transaction_amount'] = pd.to_numeric(query_df['total_transaction_amount'],errors='coerce')
    top_5_states = query_df.nlargest(5, 'total_transaction_amount')
    fig1 = px.bar(top_5_states,x='States',y='total_transaction_amount',title='Top 5 States by Transaction Amount',
                  color_discrete_sequence=px.colors.sequential.Viridis)
    fig1.show()
    bottom_5_states = query_df.nsmallest(5, 'total_transaction_amount')
    fig2 = px.bar(bottom_5_states,x='States',y='total_transaction_amount',title='Bottom 5 States by Transaction Amount',
                    color_discrete_sequence=px.colors.sequential.Bluered_r) 
    fig2.show()

In [ ]:
compare_top_vs_bottom_states()

In [ ]:
Map_user[Map_user['Years'] == 2019]

In [ ]:
# problem 5 query 1
# Total registered users per state

def total_registered_users_per_state():
    cursor.execute(f""" select States, SUM(Registeredusers) AS total_registered_users
                   from map_user
                   group by States
                   order by total_registered_users desc""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','total_registered_users'])
    fig = px.bar(
                query_df.sort_values("total_registered_users", ascending=True),  # sort for cleaner horizontal bars
                x="total_registered_users",
                y="States",
                orientation="h",
                title=f"Total Registered Users per State",
                text="total_registered_users",  # show values on bars
                color="total_registered_users",  # add gradient coloring
            )
    fig.show()
    return query_df

def total_registered_users_selected_state(state):
    cursor.execute(f"""select States,Years, SUM(Registeredusers) AS total_registered_users
                   from map_user
                   where States='{state}'
                   group by States,Years
                   order by total_registered_users desc""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','Years','total_registered_users'])
    fig = px.bar(
                query_df.sort_values("total_registered_users", ascending=True),  # sort for cleaner horizontal bars
                x="total_registered_users",
                y="Years",
                orientation="h",
                title=f"Total Registered Users in {state}",
                text="total_registered_users",  # show values on bars
                color="total_registered_users",  # add gradient coloring
            )
    fig.show()
    return query_df

In [ ]:
total_registered_users_selected_state('Maharashtra')

In [ ]:
total_registered_users_per_state()

In [ ]:
# problem 5 query 2
# App opens vs registered users

# SELECT States, SUM(RegisteredUsers) AS users, SUM(AppOpens) AS opens,
#        SUM(AppOpens)/SUM(RegisteredUsers) AS engagement_ratio
# FROM map_user
# GROUP BY States;
def app_opens_vs_registered_users():
    cursor.execute(f""" select States, SUM(RegisteredUsers) AS users, SUM(AppOpens) AS opens,
                   SUM(AppOpens)/SUM(RegisteredUsers) AS engagement_ratio
                   FROM map_user
                   GROUP BY States
                   order by engagement_ratio desc""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','users','opens','engagement_ratio'])
    fig = px.bar(
                query_df.sort_values("engagement_ratio", ascending=True),  # sort for cleaner horizontal bars
                x="engagement_ratio",
                y="States",
                orientation="h",
                title=f"App Opens vs Registered Users (Engagement Ratio)",
                text="engagement_ratio",  # show values on bars
                color="engagement_ratio",  # add gradient coloring
            )
    fig.show()
    return query_df

def engagement_ratio_yearly_state(state):
    cursor.execute(f""" select States,Years, SUM(RegisteredUsers) AS users, SUM(AppOpens) AS opens,
                   SUM(AppOpens)/SUM(RegisteredUsers) AS engagement_ratio
                   FROM map_user
                   where States='{state}'
                   GROUP BY States,Years
                   order by engagement_ratio desc""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','Years','users','opens','engagement_ratio'])
    fig = px.bar(
                query_df.sort_values("engagement_ratio", ascending=True),  # sort for cleaner horizontal bars
                x="engagement_ratio",
                y="Years",
                orientation="h",
                title=f"Engagement Ratio in {state}",
                text="engagement_ratio",  # show values on bars
                color="engagement_ratio",  # add gradient coloring
            )
    fig.show()
    return query_df


In [ ]:
app_opens_vs_registered_users()

In [ ]:
engagement_ratio_yearly_state('Tamil Nadu')

In [ ]:
# problem 5 query 3
#  Districts with highest engagement

def overall_highest_engagement_district():
    cursor.execute(f""" select States,Districts, SUM(RegisteredUsers) AS users, SUM(AppOpens) AS opens,
                   SUM(AppOpens)/SUM(RegisteredUsers) AS engagement_ratio
                   FROM map_user
                   GROUP BY States,Districts
                   order by engagement_ratio desc""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','Districts','users','opens','engagement_ratio'])
    # top 5 districts
    query_df['engagement_ratio'] = pd.to_numeric(query_df['engagement_ratio'], errors='coerce')
    top_5_df = query_df.nlargest(5, 'engagement_ratio')
    fig = px.bar(
                top_5_df.sort_values("engagement_ratio", ascending=True),  # sort for cleaner horizontal bars
                x="engagement_ratio",
                y="Districts",
                orientation="h",
                title=f"Districts with Highest Engagement Overall",
                text="engagement_ratio",  # show values on bars
                color="engagement_ratio",  # add gradient coloring
            )
    fig.show()
    
    # bottom 5 districts
    bottom_5_df = query_df.nsmallest(5, 'engagement_ratio')
    fig2 = px.bar(
                bottom_5_df.sort_values("engagement_ratio", ascending=True),  # sort for cleaner horizontal bars
                x="engagement_ratio",
                y="Districts",
                orientation="h",
                title=f"Districts with Lowest Engagement Overall",
                text="engagement_ratio",  # show values on bars
                color="engagement_ratio",  # add gradient coloring
            )       
    fig2.show()

    return query_df


def districts_with_highest_engagement(state):
    cursor.execute(f""" select States,Districts, SUM(RegisteredUsers) AS users, SUM(AppOpens) AS opens,
                   SUM(AppOpens)/SUM(RegisteredUsers) AS engagement_ratio
                   FROM map_user
                   where States='{state}'
                   GROUP BY States,Districts
                   order by engagement_ratio desc""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','Districts','users','opens','engagement_ratio'])
    # top 5 districts
    query_df['engagement_ratio'] = pd.to_numeric(query_df['engagement_ratio'], errors='coerce')
    top_5_df = query_df.nlargest(5, 'engagement_ratio')
    fig = px.bar(
                top_5_df.sort_values("engagement_ratio", ascending=True),  # sort for cleaner horizontal bars
                x="engagement_ratio",
                y="Districts",
                orientation="h",
                title=f"Districts with Highest Engagement in {state}",
                text="engagement_ratio",  # show values on bars
                color="engagement_ratio",  # add gradient coloring
            )
    fig.show()
    # bottom 5 districts
    bottom_5_df = query_df.nsmallest(5, 'engagement_ratio')
    fig2 = px.bar(
                bottom_5_df.sort_values("engagement_ratio", ascending=True),  # sort for cleaner horizontal bars
                x="engagement_ratio",
                y="Districts",
                orientation="h",
                title=f"Districts with Lowest Engagement in {state}",
                text="engagement_ratio",  # show values on bars
                color="engagement_ratio",  # add gradient coloring
            )       
    fig2.show()

    return query_df

In [ ]:
overall_highest_engagement_district()

In [ ]:
districts_with_highest_engagement('Maharashtra')

In [ ]:
# problem 5 query 4
# Growth trend over years
# 1st we shown the trend of given year for asked state
# 2nd we shown the trend of 2025-2027
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

def growth_trend_over_years(state):
    cursor.execute(f""" select states,years,sum(RegisteredUsers) as total_usr
                   from map_user
                    where states='{state}'
                   group by states,years
                   order by states,years""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['states','years','total_usr'])
    fig = px.line(query_df, x='years', y='total_usr', color='states', title='Growth Trend of Registered Users Over Years by State')
    fig.show()
    return query_df


def forecast_trend_Regusers(state,forecast_years=3):

    cursor.execute(f"""  SELECT states, years, SUM(RegisteredUsers) AS total_usr
                            FROM map_user
                            WHERE states='{state}'
                            GROUP BY states, years
                            ORDER BY years
                        """)
    
    conn.commit()
    query=cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['states','years','total_usr'])
    query_df['years'] = pd.to_numeric(query_df['years'], errors='coerce')
    query_df['total_usr'] = pd.to_numeric(query_df['total_usr'], errors='coerce')

    query_df['YoY_growth_pct']=query_df['total_usr'].pct_change()*100
    
    # forecast
    if forecast_years>0:
        series=query_df.set_index('years')['total_usr']
        series = series.astype(float)
        series.index = pd.PeriodIndex(series.index, freq='Y')
        model=ARIMA(series,order=(1,1,1))
        model_fit=model.fit()
        forecast=model_fit.forecast(steps=forecast_years)
        future_years=[query_df['years'].iloc[-1]+i for i in range(1,forecast_years+1)]
        forecast_df = pd.DataFrame({
            "states": state,
            "years": future_years,
            "total_usr": forecast,
            "YoY_growth_pct": np.nan  # can calculate vs last actual if needed
        })

        query_df = pd.concat([query_df, forecast_df], ignore_index=True)

            # --- Step 4: Plot growth trend ---
        fig = px.line(
            query_df,
            x='years',
            y='total_usr',
            color='states',
            markers=True,
            title=f'Growth Trend of Registered Users in {state}'
        )
        fig.show()
        forecast_only = query_df[query_df['years'].isin(future_years)]
        fig2 = px.bar(
            forecast_only,
            x='years',
            y='total_usr',
            color='states',
            text='total_usr',
            orientation='v',  # vertical bar is usually clearer for years
            title=f'Forecasted Registered Users in {state} (2025–2027)'
        )
        fig2.show()

        return query_df





In [ ]:
growth_trend_over_years('Tamil Nadu')

In [ ]:
# problem 5 query 5
#  Underperforming states (low app opens)

def underperforming_states_low_app_opens():
    cursor.execute(f""" select States, SUM(AppOpens) AS total_app_opens
                   from map_user
                   group by States
                   order by total_app_opens""")
    conn.commit()
    query=cursor.fetchall()
    query_df=pd.DataFrame(query,columns=['States','total_app_opens'])
    query_df['total_app_opens'] = pd.to_numeric(query_df['total_app_opens'],errors='coerce')
    fig = px.bar(
                query_df.sort_values("total_app_opens", ascending=False),  # sort for cleaner horizontal bars
                x="total_app_opens",
                y="States",
                orientation="h",
                title=f"Underperforming States (Low App Opens < 100,000)",
                text="total_app_opens",  # show values on bars
                color="total_app_opens",  # add gradient coloring
            )
    fig.show()
    bottom_df =query_df.nsmallest(5,'total_app_opens')
    fig2=px.bar(
        bottom_df.sort_values("total_app_opens", ascending=True),
        x="total_app_opens",
        y="States",
        orientation="h",
        title="Bottom 5 States by App Opens",
        text="total_app_opens",
        color="total_app_opens",
    )
    fig2.show()
    cursor.execute(f''' SELECT states, SUM(AppOpens)/SUM(RegisteredUsers) AS engagement_ratio
                        FROM map_user
                        GROUP BY states
                        having engagement_ratio < 28
                        ORDER BY engagement_ratio 
                        ''')
    conn.commit()
    query2=cursor.fetchall()
    query2_df=pd.DataFrame(query2,columns=['States','engagement_ratio'])
    query2_df['engagement_ratio'] = pd.to_numeric(query2_df['engagement_ratio'],errors='coerce')
    bottom_df2 =query2_df.nsmallest(5,'engagement_ratio')
    fig3=px.bar(
        bottom_df2.sort_values("engagement_ratio", ascending=True),
        x="engagement_ratio",
        y="States",
        orientation="h",
        title="Bottom 5 States by Engagement Ratio",
        text="engagement_ratio",
        color="engagement_ratio",
    )
    fig3.show()


In [ ]:
underperforming_states_low_app_opens()


In [ ]:
quarter_with_peak_transactions_state('Maharashtra')

In [ ]:
quarter_with_peak_transactions()

In [ ]:

Aggre_user

In [ ]:
Map_user

In [ ]:
Top_user

## Helper codes

In [ ]:
# Helper function for formatting currency
def format_currency(value):
    """Format large numbers with appropriate suffixes (T, B, M, K)"""
    if pd.isna(value):
        return "N/A"
    
    if abs(value) >= 1e12:  # Trillion
        return f"₹{value/1e12:.2f}T"
    elif abs(value) >= 1e9:  # Billion
        return f"₹{value/1e9:.2f}B"
    elif abs(value) >= 1e6:  # Million
        return f"₹{value/1e6:.2f}M"
    elif abs(value) >= 1e3:  # Thousand
        return f"₹{value/1e3:.2f}K"
    else:
        return f"₹{value:.2f}"

# Alternative version with even better formatting
def fastest_growing_state_yoy_growth_enhanced():
    # YoY Growth Percentage
    cursor.execute("""select Years, 
                   (SUM(Transaction_amount) - LAG(SUM(Transaction_amount)) OVER (ORDER BY Years)) / LAG(SUM(Transaction_amount)) OVER (ORDER BY Years) * 100 AS growth_percentage_amount
                   from aggregated_transaction
                   group by Years
                   order by Years""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','growth_percentage_amount'])
    
    # Convert to numeric to handle Decimal types
    query_df['growth_percentage_amount'] = pd.to_numeric(query_df['growth_percentage_amount'], errors='coerce')
    
    # Format growth percentage for display
    query_df['Growth_Text'] = query_df['growth_percentage_amount'].apply(lambda x: f"{x:.1f}%" if pd.notna(x) else "N/A")
    
    fig1 = px.bar(
                query_df.dropna().sort_values("growth_percentage_amount", ascending=True),
                x="growth_percentage_amount",
                y="Years",
                orientation="h",
                title="Year-over-Year Growth Percentage of Transaction Amount",
                text="Growth_Text",
                color="growth_percentage_amount",
                color_continuous_scale="RdYlBu_r"
            )
    fig1.update_traces(textposition='outside')
    fig1.update_layout(xaxis_title="Growth Percentage (%)")
    fig1.show()

    # Total Transaction Amount with better formatting
    cursor.execute(""" Select Years,
                   sum(Transaction_amount) as Total_Trans_amt
                   from aggregated_transaction
                   group by Years
                   order by Years""")
    conn.commit()
    query = cursor.fetchall()
    query_df = pd.DataFrame(query,columns=['Years','Total_Trans_amt'])
    
    # Convert to numeric to handle Decimal types
    query_df['Total_Trans_amt'] = pd.to_numeric(query_df['Total_Trans_amt'], errors='coerce')
    
    # Create formatted amounts in Trillions for easier reading
    query_df['Amount_in_Trillions'] = query_df['Total_Trans_amt'] / 1e12
    query_df['Formatted_Amount'] = query_df['Total_Trans_amt'].apply(format_currency)
    
    fig2 = px.bar(
                query_df,
                x="Amount_in_Trillions",
                y="Years",
                orientation="h",
                title="Yearly Transaction Amount Growth (in Trillions ₹)",
                text="Formatted_Amount",
                color="Amount_in_Trillions",
                color_continuous_scale="Viridis"
            )
    
    fig2.update_traces(textposition='outside')
    fig2.update_layout(xaxis_title="Transaction Amount (Trillions ₹)")
    fig2.show()
    
    # Print summary table for reference
    print("\nYearly Transaction Summary:")
    print("-" * 50)
    for _, row in query_df.iterrows():
        print(f"Year {int(row['Years'])}: {row['Formatted_Amount']}")

# Run the enhanced version
fastest_growing_state_yoy_growth_enhanced()

In [ ]:
Aggre_transaction

In [ ]:
Top_transaction

In [ ]:
Map_transaction